In [48]:
import tensorflow as tf
from tensorflow import keras
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import pandas as pd
import os

In [49]:
checkpoint_path = "model/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

In [50]:
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1, monitor='accuracy',
                                                 mode='max',
                                                 save_best_only=True)

In [51]:
_labels_ = ['STANDING', 'WALKING_UPSTAIRS', 'LAYING', 'WALKING', 'SITTING', 'WALKING_DOWNSTAIRS']

In [52]:
def label_to_vector(label):
    return [int(label == l) for l in _labels_]

In [53]:
def vector_to_label(vec):
    return _labels_[vec.index(1)]

In [54]:
def get_Y(labels):
    return [label_to_vector(l[0]) for l in labels]

In [55]:
# build model: 66 -> 133 -> 180 -> 60 -> 6
def create_model():
    model = tf.keras.models.Sequential([
        keras.layers.Dense(150, input_dim=66, activation="relu"),
        keras.layers.Dense(180, activation="relu"),
        keras.layers.Dense(80, activation="relu"),
        keras.layers.Dense(6, activation="softmax")
    ])
    return model

In [56]:
df = pd.read_csv('data/df_grouped.csv')
pd.set_option('display.max_columns', None)

In [57]:
shuffled = shuffle(df)
shuffled

,Subject,Activity,fBodyAcc-std()-X,fBodyAcc-std()-Y,fBodyAcc-std()-Z,fBodyAccJerk-std()-X,fBodyAccJerk-std()-Y,fBodyAccJerk-std()-Z,fBodyAccMag-std(),fBodyBodyAccJerkMag-std(),fBodyBodyGyroJerkMag-std(),fBodyBodyGyroMag-std(),fBodyGyro-std()-X,fBodyGyro-std()-Y,fBodyGyro-std()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAccJerk-std()-X,tBodyAccJerk-std()-Y,tBodyAccJerk-std()-Z,tBodyAccJerkMag-std(),tBodyAccMag-std(),tBodyGyro-std()-X,tBodyGyro-std()-Y,tBodyGyro-std()-Z,tBodyGyroJerk-std()-X,tBodyGyroJerk-std()-Y,tBodyGyroJerk-std()-Z,tBodyGyroJerkMag-std(),tBodyGyroMag-std(),tGravityAcc-std()-X,tGravityAcc-std()-Y,tGravityAcc-std()-Z,tGravityAccMag-std(),fBodyAcc-mean()-X,fBodyAcc-mean()-Y,fBodyAcc-mean()-Z,fBodyAccJerk-mean()-X,fBodyAccJerk-mean()-Y,fBodyAccJerk-mean()-Z,fBodyAccMag-mean(),fBodyBodyAccJerkMag-mean(),fBodyBodyGyroJerkMag-mean(),fBodyBodyGyroMag-mean(),fBodyGyro-mean()-X,fBodyGyro-mean()-Y,fBodyGyro-mean()-Z,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAccJerk-mean()-X,tBodyAccJerk-mean()-Y,tBodyAccJerk-mean()-Z,tBodyAccJerkMag-mean(),tBodyAccMag-mean(),tBodyGyro-mean()-X,tBodyGyro-mean()-Y,tBodyGyro-mean()-Z,tBodyGyroJerk-mean()-X,tBodyGyroJerk-mean()-Y,tBodyGyroJerk-mean()-Z,tBodyGyroJerkMag-mean(),tBodyGyroMag-mean(),tGravityAcc-mean()-X,tGravityAcc-mean()-Y,tGravityAcc-mean()-Z,tGravityAccMag-mean()
68,12,STANDING,0.784424,-1.355893,4.185520,-4.052750,0.320336,2.343525,-0.876967,-3.012325,-6.019324,4.819233,3.082074,0.805406,4.686545,6.340581,3.878778,3.396559,6.290296,-0.575160,3.911205,2.332678,1.544031,5.155827,5.551914,1.772659,-4.129246,-0.586058,0.852467,-2.649899,5.601318,3.712069,1.391944,4.055501,1.924423,-7.609411,-1.011042,6.365923,4.658593,1.723454,-1.636638,7.463978,3.335853,-0.336333,1.624002,1.513452,2.530298,-5.480215,3.992460,-0.960641,2.872867,0.770308,3.540804,1.776148,5.737229,3.585967,-2.060529,3.154509,-2.208789,1.118703,0.490268,2.308488,2.663020,0.238618,2.266921,2.975360,4.276553,-0.938757
158,27,STANDING,0.795336,-1.339076,4.199993,-4.036341,0.294965,2.356388,-0.847459,-3.014415,-6.016811,4.843822,3.263234,0.811188,4.703937,6.344954,3.879980,3.429172,6.305057,-0.549886,3.922004,2.320339,1.584643,5.312552,5.694860,1.789143,-4.151142,-0.590017,0.852439,-2.663561,5.780707,3.729904,1.384323,4.090336,1.977848,-7.613145,-1.017356,6.348445,4.676996,1.756067,-1.618167,7.494167,3.336166,-0.319382,1.594262,1.464210,2.532968,-5.518225,3.996770,-0.957524,2.877123,0.770249,3.541696,1.786479,5.758755,3.570329,-2.063309,3.147725,-2.184479,1.117953,0.490969,2.316062,2.680545,0.337783,2.249298,2.911829,4.010997,-0.941110
41,7,WALKING_UPSTAIRS,-0.378863,-2.483714,3.763784,-5.052039,1.213717,1.791828,-2.144774,-2.920153,-6.057605,4.698655,1.547938,0.741861,4.007768,5.946353,3.794357,2.430211,5.506939,-1.441398,3.520586,2.811941,-0.280430,3.792663,3.912496,1.136320,-3.727993,-0.513600,0.853269,-2.446222,4.629835,3.569520,1.310698,3.998179,-0.246214,-7.354998,-0.676141,6.983752,3.718747,0.608128,-2.229154,6.350755,3.325019,-0.585307,1.829737,1.949249,2.495279,-4.063039,4.007910,-0.995898,2.884312,0.868751,3.530974,1.775184,4.850028,4.228548,-2.067654,3.148717,-2.108040,1.145271,0.444451,2.822443,2.319572,-0.919904,2.564661,2.968323,6.046571,-0.841782
101,17,WALKING_UPSTAIRS,-0.969192,-3.181129,3.769668,-5.524104,1.756451,1.832408,-2.662988,-2.903036,-6.055615,4.696572,1.775407,0.724673,3.973163,5.758157,3.743334,2.476513,5.212305,-1.927470,3.563380,2.923252,-1.080846,3.906116,3.522146,1.099083,-3.654979,-0.507169,0.853706,-2.433308,4.557220,3.572027,1.214495,4.051603,-1.200694,-7.256358,-0.475450,6.909516,3.449873,0.025365,-2.145020,5.795236,3.322204,-0.618721,1.867146,2.026925,2.489107,-3.936155,3.994561,-0.978272,2.897480,1.011207,3.554029,1.865528,4.533984,4.473946,-2.055518,3.097171,-2.056691,1.079467,0.351092,2.254530,2.234879,-1.099617,2.469671,3.094893,5.428565,-0.805145
71,12,WALKING_UPSTAIRS,-0.427146,-2.964487,3.903621,-5.199914,1.643679,1.942439,-2.118749,-2.90018

In [58]:
X = shuffled[shuffled.columns[2:]].values.tolist()
Y_labels = shuffled[shuffled.columns[1:2]].values.tolist()

In [59]:
Y = get_Y(Y_labels)
len(Y), len(X)

(180, 180)

In [60]:
model = create_model()

In [61]:
# compile
model.compile(optimizer="adam", loss="MSE", metrics=['accuracy'])

In [62]:
# train
model.fit(X, Y, epochs=1200, callbacks=[cp_callback])

Epoch 1/2000
1/6 [====>.........................] - ETA: 0s - loss: 0.1502 - accuracy: 0.1875
Epoch 00001: accuracy improved from -inf to 0.15556, saving model to model\cp.ckpt
6/6 [==============================] - 0s 3ms/step - loss: 0.1422 - accuracy: 0.1556
Epoch 2/2000
1/6 [====>.........................] - ETA: 0s - loss: 0.1177 - accuracy: 0.5312
Epoch 00002: accuracy improved from 0.15556 to 0.48889, saving model to model\cp.ckpt
6/6 [==============================] - 0s 4ms/step - loss: 0.1159 - accuracy: 0.4889
Epoch 3/2000
1/6 [====>.........................] - ETA: 0s - loss: 0.0986 - accuracy: 0.7188
Epoch 00003: accuracy improved from 0.48889 to 0.56667, saving model to model\cp.ckpt
6/6 [==============================] - 0s 4ms/step - loss: 0.1004 - accuracy: 0.5667
Epoch 4/2000
1/6 [====>.........................] - ETA: 0s - loss: 0.1024 - accuracy: 0.5000
Epoch 00004: accuracy improved from 0.56667 to 0.57778, saving model to model\cp.ckpt
6/6 [=======================

Epoch 35/2000
1/6 [====>.........................] - ETA: 0s - loss: 0.0474 - accuracy: 0.7500
Epoch 00035: accuracy did not improve from 0.91111
6/6 [==============================] - 0s 833us/step - loss: 0.0310 - accuracy: 0.8667
Epoch 36/2000
1/6 [====>.........................] - ETA: 0s - loss: 0.0335 - accuracy: 0.8750
Epoch 00036: accuracy did not improve from 0.91111
6/6 [==============================] - 0s 833us/step - loss: 0.0397 - accuracy: 0.8500
Epoch 37/2000
1/6 [====>.........................] - ETA: 0s - loss: 0.0264 - accuracy: 0.9062
Epoch 00037: accuracy did not improve from 0.91111
6/6 [==============================] - 0s 666us/step - loss: 0.0316 - accuracy: 0.8833
Epoch 38/2000
1/6 [====>.........................] - ETA: 0s - loss: 0.0437 - accuracy: 0.8125
Epoch 00038: accuracy did not improve from 0.91111
6/6 [==============================] - 0s 833us/step - loss: 0.0328 - accuracy: 0.8778
Epoch 39/2000
1/6 [====>.........................] - ETA: 0s - loss:

Epoch 70/2000
1/6 [====>.........................] - ETA: 0s - loss: 0.0126 - accuracy: 0.9688
Epoch 00070: accuracy did not improve from 0.94444
6/6 [==============================] - 0s 1ms/step - loss: 0.0187 - accuracy: 0.9278
Epoch 71/2000
1/6 [====>.........................] - ETA: 0s - loss: 0.0261 - accuracy: 0.8750
Epoch 00071: accuracy did not improve from 0.94444
6/6 [==============================] - 0s 1ms/step - loss: 0.0196 - accuracy: 0.9222
Epoch 72/2000
1/6 [====>.........................] - ETA: 0s - loss: 0.0127 - accuracy: 0.9062
Epoch 00072: accuracy did not improve from 0.94444
6/6 [==============================] - 0s 1000us/step - loss: 0.0182 - accuracy: 0.9278
Epoch 73/2000
1/6 [====>.........................] - ETA: 0s - loss: 0.0285 - accuracy: 0.8750
Epoch 00073: accuracy did not improve from 0.94444
6/6 [==============================] - 0s 833us/step - loss: 0.0206 - accuracy: 0.9111
Epoch 74/2000
1/6 [====>.........................] - ETA: 0s - loss: 0.

1/6 [====>.........................] - ETA: 0s - loss: 0.0041 - accuracy: 1.0000
Epoch 00105: accuracy did not improve from 0.96111
6/6 [==============================] - 0s 833us/step - loss: 0.0131 - accuracy: 0.9500
Epoch 106/2000
1/6 [====>.........................] - ETA: 0s - loss: 0.0068 - accuracy: 1.0000
Epoch 00106: accuracy did not improve from 0.96111
6/6 [==============================] - 0s 1ms/step - loss: 0.0167 - accuracy: 0.9278
Epoch 107/2000
1/6 [====>.........................] - ETA: 0s - loss: 0.0185 - accuracy: 0.9375
Epoch 00107: accuracy did not improve from 0.96111
6/6 [==============================] - 0s 834us/step - loss: 0.0162 - accuracy: 0.9444
Epoch 108/2000
1/6 [====>.........................] - ETA: 0s - loss: 0.0144 - accuracy: 0.9688
Epoch 00108: accuracy did not improve from 0.96111
6/6 [==============================] - 0s 667us/step - loss: 0.0163 - accuracy: 0.9389
Epoch 109/2000
1/6 [====>.........................] - ETA: 0s - loss: 0.0113 - ac

1/6 [====>.........................] - ETA: 0s - loss: 0.0103 - accuracy: 0.9688
Epoch 00140: accuracy did not improve from 0.96111
6/6 [==============================] - 0s 833us/step - loss: 0.0128 - accuracy: 0.9500
Epoch 141/2000
1/6 [====>.........................] - ETA: 0s - loss: 0.0275 - accuracy: 0.8438
Epoch 00141: accuracy did not improve from 0.96111
6/6 [==============================] - 0s 833us/step - loss: 0.0197 - accuracy: 0.9222
Epoch 142/2000
1/6 [====>.........................] - ETA: 0s - loss: 0.0068 - accuracy: 0.9688
Epoch 00142: accuracy did not improve from 0.96111
6/6 [==============================] - 0s 833us/step - loss: 0.0155 - accuracy: 0.9444
Epoch 143/2000
1/6 [====>.........................] - ETA: 0s - loss: 0.0187 - accuracy: 0.9375
Epoch 00143: accuracy did not improve from 0.96111
6/6 [==============================] - 0s 832us/step - loss: 0.0178 - accuracy: 0.9333
Epoch 144/2000
1/6 [====>.........................] - ETA: 0s - loss: 0.0053 - 

...

Epoch 1855/2000
1/6 [====>.........................] - ETA: 0s - loss: 0.0017 - accuracy: 1.0000
Epoch 01855: accuracy did not improve from 1.00000
6/6 [==============================] - 0s 500us/step - loss: 0.0188 - accuracy: 0.9389
Epoch 1856/2000
1/6 [====>.........................] - ETA: 0s - loss: 0.0511 - accuracy: 0.8438
Epoch 01856: accuracy did not improve from 1.00000
6/6 [==============================] - 0s 1000us/step - loss: 0.0144 - accuracy: 0.9556
Epoch 1857/2000
1/6 [====>.........................] - ETA: 0s - loss: 3.4505e-04 - accuracy: 1.0000
Epoch 01857: accuracy did not improve from 1.00000
6/6 [==============================] - 0s 833us/step - loss: 0.0106 - accuracy: 0.9556
Epoch 1858/2000
1/6 [====>.........................] - ETA: 0s - loss: 0.0076 - accuracy: 0.9688
Epoch 01858: accuracy did not improve from 1.00000
6/6 [==============================] - 0s 1ms/step - loss: 0.0111 - accuracy: 0.9611
Epoch 1859/2000
1/6 [====>.........................] - ET

Epoch 1890/2000
1/6 [====>.........................] - ETA: 0s - loss: 0.0026 - accuracy: 1.0000
Epoch 01890: accuracy did not improve from 1.00000
6/6 [==============================] - 0s 1000us/step - loss: 0.0019 - accuracy: 0.9944
Epoch 1891/2000
1/6 [====>.........................] - ETA: 0s - loss: 1.2061e-04 - accuracy: 1.0000
Epoch 01891: accuracy did not improve from 1.00000
6/6 [==============================] - 0s 668us/step - loss: 0.0029 - accuracy: 0.9889
Epoch 1892/2000
1/6 [====>.........................] - ETA: 0s - loss: 0.0025 - accuracy: 1.0000
Epoch 01892: accuracy did not improve from 1.00000
6/6 [==============================] - 0s 833us/step - loss: 0.0025 - accuracy: 0.9944
Epoch 1893/2000
1/6 [====>.........................] - ETA: 0s - loss: 0.0109 - accuracy: 0.9375
Epoch 01893: accuracy did not improve from 1.00000
6/6 [==============================] - 0s 833us/step - loss: 0.0028 - accuracy: 0.9889
Epoch 1894/2000
1/6 [====>.........................] - 

Epoch 1925/2000
1/6 [====>.........................] - ETA: 0s - loss: 4.9389e-04 - accuracy: 1.0000
Epoch 01925: accuracy did not improve from 1.00000
6/6 [==============================] - 0s 1ms/step - loss: 0.0056 - accuracy: 0.9722
Epoch 1926/2000
1/6 [====>.........................] - ETA: 0s - loss: 0.0015 - accuracy: 1.0000
Epoch 01926: accuracy did not improve from 1.00000
6/6 [==============================] - 0s 833us/step - loss: 0.0049 - accuracy: 0.9778
Epoch 1927/2000
1/6 [====>.........................] - ETA: 0s - loss: 0.0018 - accuracy: 1.0000
Epoch 01927: accuracy did not improve from 1.00000
6/6 [==============================] - 0s 667us/step - loss: 0.0050 - accuracy: 0.9722
Epoch 1928/2000
1/6 [====>.........................] - ETA: 0s - loss: 0.0202 - accuracy: 0.8750
Epoch 01928: accuracy did not improve from 1.00000
6/6 [==============================] - 0s 1000us/step - loss: 0.0059 - accuracy: 0.9722
Epoch 1929/2000
1/6 [====>.........................] - ET

Epoch 1960/2000
1/6 [====>.........................] - ETA: 0s - loss: 1.0828e-04 - accuracy: 1.0000
Epoch 01960: accuracy did not improve from 1.00000
6/6 [==============================] - 0s 834us/step - loss: 0.0017 - accuracy: 0.9944
Epoch 1961/2000
1/6 [====>.........................] - ETA: 0s - loss: 4.9273e-05 - accuracy: 1.0000
Epoch 01961: accuracy did not improve from 1.00000
6/6 [==============================] - 0s 667us/step - loss: 0.0018 - accuracy: 0.9944
Epoch 1962/2000
1/6 [====>.........................] - ETA: 0s - loss: 0.0095 - accuracy: 0.9688
Epoch 01962: accuracy did not improve from 1.00000
6/6 [==============================] - 0s 833us/step - loss: 0.0079 - accuracy: 0.9667
Epoch 1963/2000
1/6 [====>.........................] - ETA: 0s - loss: 0.0013 - accuracy: 1.0000
Epoch 01963: accuracy did not improve from 1.00000
6/6 [==============================] - 0s 667us/step - loss: 0.0064 - accuracy: 0.9778
Epoch 1964/2000
1/6 [====>.........................]

Epoch 1995/2000
1/6 [====>.........................] - ETA: 0s - loss: 5.5065e-04 - accuracy: 1.0000
Epoch 01995: accuracy did not improve from 1.00000
6/6 [==============================] - 0s 834us/step - loss: 0.0021 - accuracy: 0.9944
Epoch 1996/2000
1/6 [====>.........................] - ETA: 0s - loss: 0.0011 - accuracy: 1.0000
Epoch 01996: accuracy did not improve from 1.00000
6/6 [==============================] - 0s 832us/step - loss: 0.0066 - accuracy: 0.9667
Epoch 1997/2000
1/6 [====>.........................] - ETA: 0s - loss: 0.0112 - accuracy: 0.9375
Epoch 01997: accuracy did not improve from 1.00000
6/6 [==============================] - 0s 833us/step - loss: 0.0114 - accuracy: 0.9500
Epoch 1998/2000
1/6 [====>.........................] - ETA: 0s - loss: 0.0079 - accuracy: 0.9688
Epoch 01998: accuracy did not improve from 1.00000
6/6 [==============================] - 0s 666us/step - loss: 0.0085 - accuracy: 0.9611
Epoch 1999/2000
1/6 [====>.........................] - E

In [63]:
# Loads the weights
model.load_weights(checkpoint_path)

In [64]:
df_clean = pd.read_csv('data/df_no_nan.csv')

In [65]:
X_2 = df_clean[df_clean.columns[1:]].values.tolist()
Y_2_labels = df_clean[df_clean.columns[:1]].values.tolist()

In [66]:
Y_2 = get_Y(Y_2_labels)
len(Y_2), len(X_2)

(9826, 9826)

In [67]:
# evaluation
print("\n model evaluation:")
model.evaluate(X_2, Y_2, verbose=1)


 model evaluation:
308/308 [==============================] - 0s 409us/step - loss: 0.0207 - accuracy: 0.9149


[0.020657282322645187, 0.9149196147918701]

In [119]:
# make prediction
# predictions = model.predict(data)